In [2]:
import requests
import json
import pandas as pd

API_KEY = '8e87c18e53abc6e463fb7835b677128e'
USER_AGENT = 'Dataquest'

headers = {
    'user-agent': USER_AGENT
}

Now we create some functions to call last_fm api

In [3]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def user_friend(user):
    r = lastfm_get({
        'method': 'user.getFriends',
        'user' : user
        })
    return r.json()['friends']['user']

def user_fav_track(user):
    r = lastfm_get({
        'method' : 'user.getLovedTracks',
        'user': user
    })
    return r.json()['lovedtracks']['track']

def user_recent_track(user):
    r = lastfm_get({
        'method' : 'user.getRecentTracks',
        'user': user
    })
    return r.json()['recenttracks']['track']

Now we use the calls to add more users by using user friends

In [4]:
user_list = ['Efoja_87', 'Crybkeeper', 'epi_gee', 'meehawl', 'bisa', 'tYhJyYsRuLeS', 'figlt', 'Zescher', 'donghaerang', 'Jd_Jd_Jd', 'temujin1234', 'sibonforever', 'tweeglitch', 'novira', 'AimsterSkitz0rz', 'NorseDave', '40belowsummer']

user_friends = []

for i in user_list:
    u_friend = user_friend(i)
    for j in u_friend:
        user_friends.append(j)

user_df = pd.DataFrame(user_friends)
print(user_df.head(5))

       name                                url      country playlists  \
0   jonnyed   https://www.last.fm/user/jonnyed       Norway         0   
1    Alainn    https://www.last.fm/user/Alainn  Netherlands         0   
2  Nutsh3ll  https://www.last.fm/user/Nutsh3ll        Italy         0   
3  shatelei  https://www.last.fm/user/shatelei      Ukraine         0   
4    doma20    https://www.last.fm/user/doma20         None         0   

  playcount                                              image  \
0         0  [{'size': 'small', '#text': 'https://lastfm.fr...   
1         0  [{'size': 'small', '#text': 'https://lastfm.fr...   
2         0  [{'size': 'small', '#text': 'https://lastfm.fr...   
3         0  [{'size': 'small', '#text': 'https://lastfm.fr...   
4         0  [{'size': 'small', '#text': 'https://lastfm.fr...   

                                          registered         realname  \
0  {'unixtime': '1216989550', '#text': '2008-07-2...  Jonny Edvardsen   
1  {'unixtime': '1

Now we drop duplicated users

In [5]:
print(user_df.shape)
user_df = user_df.drop_duplicates(subset='name', keep="first")
print(user_df.shape)

(832, 11)
(771, 11)


saving the data to csv

In [6]:
user_df.to_csv('users.csv')

We now use the MSD dataset

In [7]:
songs_df = pd.read_csv('song_data.csv')
user_df = pd.read_csv('triplets_file.csv')
print(songs_df.head(5))
print(user_df.head(5))

              song_id              title  \
0  SOQMMHC12AB0180CB8       Silent Night   
1  SOVFVAK12A8C1350D9        Tanssi vaan   
2  SOGTUKN12AB017F4F1  No One Could Ever   
3  SOBNYVR12A8C13558C      Si Vos Querés   
4  SOHSBXH12A8C13B0DF   Tangle Of Aspens   

                                release       artist_name  year  
0                 Monster Ballads X-Mas  Faster Pussy cat  2003  
1                           Karkuteillä  Karkkiautomaatti  1995  
2                                Butter    Hudson Mohawke  2006  
3                               De Culo       Yerba Brava  2003  
4  Rene Ablaze Presents Winter Sessions        Der Mystic     0  
                                    user_id             song_id  listen_count
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1
3  b80344d063b5ccb3212f7653

Now we use the last_fm api to add tags to the songs

In [8]:
for i, r in songs_df.head(5).iterrows():
    response = lastfm_get({
        'method' : 'track.getInfo',
        'track': r['title'],
        'artist': r['artist_name']
    })

    jprint(response.json())

{
    "track": {
        "album": {
            "artist": "Various Artists",
            "image": [
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/34s/a2eee9eb2f0d4b04bdea9a400897c5f7.png",
                    "size": "small"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/64s/a2eee9eb2f0d4b04bdea9a400897c5f7.png",
                    "size": "medium"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/174s/a2eee9eb2f0d4b04bdea9a400897c5f7.png",
                    "size": "large"
                },
                {
                    "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/a2eee9eb2f0d4b04bdea9a400897c5f7.png",
                    "size": "extralarge"
                }
            ],
            "title": "Monster Ballads Xmas",
            "url": "https://www.last.fm/music/Various+Artists/Monster+Ballads+Xmas"
        },

With this we understand that there are some calls where we get error message like this:
{
    "error": 6,
    "links": [],
    "message": "Track not found"
}
so we write a loop to work around this and add only tags

In [9]:
track_tags = []

for i, r in songs_df.head(10).iterrows():
    response = lastfm_get({
        'method' : 'track.getInfo',
        'track': r['title'],
        'artist': r['artist_name']
    })

    if "error" in response.json():
        track_tags.append({'tag': []})

    else:
        info = response.json()['track']['toptags']
        track_tags.append(info)
    
track_df = pd.DataFrame(track_tags)
print(track_df)

                                                 tag
0  [{'name': 'christmas', 'url': 'https://www.las...
1                                                 []
2  [{'name': 'Hip-Hop', 'url': 'https://www.last....
3                                                 []
4                                                 []
5                                                 []
6                                                 []
7                                                 []
8                                                 []
9                                                 []


Now we extract needed info needed from the last_fm api and add it to the songs df and test it with 10 values first 

In [10]:
track_info = []

for i, r in songs_df.head(10).iterrows():
    response = lastfm_get({
        'method' : 'track.getInfo',
        'track': r['title'],
        'artist': r['artist_name']
    })

    if "error" in response.json():
        track_info.append({'tag': [], 'image': [], 'duration': "", 'listeners': "", 'playcount': ""})

    else:
        if "album" in response.json()['track']:
            urls = response.json()['track']['album']['image']
        else:
            urls = []
        
        if "toptags" in response.json()["track"]:
            tags = response.json()['track']['toptags']['tag']
        else:
            tags = []
        
        if "duration" in response.json()["track"]:
            dr = response.json()['track']['duration']
        else:
            dr = ""
        
        if "listeners" in response.json()["track"]:
            lr = response.json()['track']['listeners']
        else:
            lr = ""
        
        if "playcount" in response.json()["track"]:
            pc = response.json()['track']['playcount']
        else:
            pc = ""

        track_info.append({'tag': tags, 'image': urls, 'duration': dr, 'listeners': lr, 'playcount': pc})

track_df = pd.DataFrame(track_info)
test_df = songs_df.head(10)
df = pd.concat([test_df, track_df], axis=1, ignore_index=False)
print(df)

              song_id                                              title  \
0  SOQMMHC12AB0180CB8                                       Silent Night   
1  SOVFVAK12A8C1350D9                                        Tanssi vaan   
2  SOGTUKN12AB017F4F1                                  No One Could Ever   
3  SOBNYVR12A8C13558C                                      Si Vos Querés   
4  SOHSBXH12A8C13B0DF                                   Tangle Of Aspens   
5  SOZVAPQ12A8C13B63C  Symphony No. 1 G minor "Sinfonie Serieuse"/All...   
6  SOQVRHI12A6D4FB2D7                                   We Have Got Love   
7  SOEYRFT12AB018936C                                  2 Da Beat Ch'yall   
8  SOPMIYT12A6D4F851E                                            Goodbye   
9  SOJCFMH12A8C13B0C2                         Mama_ mama can't you see ?   

                                release  \
0                 Monster Ballads X-Mas   
1                           Karkuteillä   
2                                B

Now we run it for the entire data

In [11]:
track_info = []

for i, r in songs_df.head(10000).iterrows():
    response = lastfm_get({
        'method' : 'track.getInfo',
        'track': r['title'],
        'artist': r['artist_name']
    })

    if "error" in response.json():
        track_info.append({'tag': [], 'image': [], 'duration': "", 'listeners': "", 'playcount': ""})

    else:
        if "album" in response.json()['track']:
            urls = response.json()['track']['album']['image']
        else:
            urls = []
        
        if "toptags" in response.json()["track"]:
            tags = response.json()['track']['toptags']['tag']
        else:
            tags = []
        
        if "duration" in response.json()["track"]:
            dr = response.json()['track']['duration']
        else:
            dr = ""
        
        if "listeners" in response.json()["track"]:
            lr = response.json()['track']['listeners']
        else:
            lr = ""
        
        if "playcount" in response.json()["track"]:
            pc = response.json()['track']['playcount']
        else:
            pc = ""

        track_info.append({'tag': tags, 'image': urls, 'duration': dr, 'listeners': lr, 'playcount': pc})

track_df = pd.DataFrame(track_info)
track_df.to_csv('track_info.csv')
temp_songs_df = songs_df.head(10000)
df = pd.concat([temp_songs_df, track_df], axis=1, ignore_index=False)
df.to_csv('added_info.csv')

In [12]:
print(track_df.shape)
print(df.shape)

(10000, 5)
(10000, 10)


In [16]:
df.describe(include=['object'])

,song_id,title,release,artist_name,tag,image,duration,listeners,playcount
count,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,10000,9777,9333,8355,4777,6505,643,4407,5560
top,SOQMMHC12AB0180CB8,Intro,Greatest Hits,A Flock Of Seagulls,[],[],0,,
freq,1,23,18,5,4908,2145,2088,566,566


In [22]:
track = pd.read_csv('added_info.csv')
track.head(2)

,Unnamed: 0,song_id,title,release,artist_name,year,tag,image,duration,listeners,playcount
0,0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,"[{'name': 'christmas', 'url': 'https://www.las...",[{'#text': 'https://lastfm.freetls.fastly.net/...,253000.0,1968.0,6601.0
1,1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,155000.0,1586.0,4996.0


In [23]:
track.describe(include=['object'])

,song_id,title,release,artist_name,tag,image
count,10000,10000,10000,10000,10000,10000
unique,10000,9777,9333,8355,4777,6505
top,SOQMMHC12AB0180CB8,Intro,Greatest Hits,A Flock Of Seagulls,[],[]
freq,1,23,18,5,4908,2145


In [24]:
df.head(2)

,song_id,title,release,artist_name,year,tag,image,duration,listeners,playcount
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,"[{'name': 'christmas', 'url': 'https://www.las...",[{'#text': 'https://lastfm.freetls.fastly.net/...,253000,1968,6601
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,155000,1586,4996


In [32]:
track_df.to_pickle('tracks_new_info.pkl')
df.to_pickle('tracks_all_info.pkl')

In [33]:
t = pd.read_pickle('track_new_info.pkl')
t.describe(include=['object'])

,tag,image,duration,listeners,playcount
count,10000,10000,10000,10000,10000
unique,4777,6505,643,4407,5560
top,[],[],0,,
freq,4908,2145,2088,566,566
